<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import nltk
from datetime import timedelta
import os
nltk.download('punkt')
from nltk.corpus import stopwords
import numpy as np
import sqlite3
from collections import Counter, defaultdict
from string import punctuation
import re
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
sw = stopwords.words('english')
import janitor
import spacy
from textblob import TextBlob
tb = TextBlob('')
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_lg')
import seaborn as sns
import scipy.stats as stats
from scipy.stats import kruskal
from gensim.models import CoherenceModel
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact_manual, interactive_output
from IPython.display import display, clear_output
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import spacy
import math
from textblob import TextBlob
tb = TextBlob('')
from spacytextblob.spacytextblob import SpacyTextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from random import randint
from sklearn.model_selection import RandomizedSearchCV
import winsound
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from spellchecker import SpellChecker
from textblob import TextBlob
import eli5
from eli5.sklearn import PermutationImportance
import warnings
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import words
from nltk.util import ngrams



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aidan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def count_ngrams(text, n):
    ### Function that takes in # of n for ngrams, and text to be processed. Then counts and sorts ngrams to 
    ### find the most common ngrams!
    tokens = word_tokenize(text)  
    n_grams = ngrams(tokens, n)
    n_gram_freq = Counter(n_grams)
    sorted_n_grams = dict(sorted(n_gram_freq.items(), key=lambda item: item[1], reverse=True))
    
    
    return sorted_n_grams

def completion_percentage(df, columns_to_process):
    ### A function that finds various completion percentages for the text data to inform the reader.
    completion_dict = {}

    for column in columns_to_process:
        total_count = len(df)
        non_empty_count = df[column].apply(lambda x: x.strip() if isinstance(x, str) else x).replace('', pd.NA).notna().sum()
        
        if total_count > 0:
            perc_completion = non_empty_count / total_count * 100
        else:
            perc_completion = 0
        
        completion_dict[column] = perc_completion
    print("Below is the percantage of entires with at least some unique text for a fire-risk grouping:")
    return completion_dict

def basic_text(df, columns_to_process):
    ### A function to compile very basic text analyses
    results = {}
    
    for column in columns_to_process:
        concatenated_text = df[column].astype(str).str.cat(sep=' ')

        text_clean = [word for word in concatenated_text.split()]
        c = Counter(text_clean)

        total_tokens = len(text_clean)
        unique_tokens = len(set(text_clean))

        lex_diversity = unique_tokens / total_tokens if total_tokens != 0 else "NA"
        avg_token_len = np.mean([len(word) for word in text_clean])
        top_10 = c.most_common(10)

        avg_tokens_per_entry = df[column].apply(lambda x: len(str(x).split())).mean()
        
        
        results[column] = {
            'total_tokens': total_tokens,
            'unique_tokens': unique_tokens,
            'average_tokens' : avg_tokens_per_entry,
            'avg_token_length': avg_token_len,
            'lexical_diversity': lex_diversity,
            'top_10': top_10,
        }
    results_df = pd.DataFrame(results)
    return results_df



In [3]:
def contains_target_word(text, target_word):
    return target_word in text

def percentage_with_target_words(data, target_words):
    percentage_with_target_words = {}

    data_copy = data.copy()

    for word in target_words:
        data_copy[f'contains_{word}'] = data_copy['combined_text'].apply(contains_target_word, target_word=word)
        percentage_with_target_words[word] = (data_copy[f'contains_{word}'].sum() / len(data_copy)) * 100
        data_copy.drop(columns=[f'contains_{word}'], inplace=True)

    sorted_results = sorted(percentage_with_target_words.items(), key=lambda x: x[1], reverse=True)

    for word, percentage in sorted_results:
        print(f"The word '{word}' appears in {percentage:.2f}% of the combined notes fields for fires.")
        
def percentage_by_risk(data, target_words):
    results_by_risk = {}

    data_copy = data.copy()

    custom_order = ['high', 'mod', 'low']

    grouped_data = data_copy.groupby('rrf_rr_desc', sort=False)  # Use sort=False to maintain custom order

    for risk, group in grouped_data:
        percentage_with_target_words = {}
        for word in target_words:
            group[f'contains_{word}'] = group['combined_text'].apply(contains_target_word, target_word=word)
            percentage_with_target_words[word] = (group[f'contains_{word}'].sum() / len(group)) * 100
            group.drop(columns=[f'contains_{word}'], inplace=True)
        results_by_risk[risk] = percentage_with_target_words

    for risk in custom_order:
        if risk in results_by_risk:
            percentages = results_by_risk[risk]
            print(f"Risk: {risk.capitalize()}")
            for word, percentage in sorted(percentages.items(), key=lambda x: x[1], reverse=True):
                print(f"The word '{word}' appears in {percentage:.2f}% of the texts for this risk.")
            print()

            
            
            
def extract_context(text, target_word, context_window_size = 5):
    words = text.split()
    target_indices = [i for i, word in enumerate(words) if word.lower() == target_word]
    
    contexts = []
    for index in target_indices:
        start_index = max(0, index - context_window_size)
        end_index = min(len(words), index + context_window_size + 1)
        context = words[start_index:end_index]
        contexts.append(context)
    
    return contexts

def extract_ordered_ngrams(context_list, n =3):
    ordered_trigrams = []
    for context in context_list:
        ordered_trigrams.extend(" ".join(context[i:i+n]) for i in range(len(context) - 2))
    return ordered_trigrams

def vectorize_text_with_ngrams(text_data, labels, ngram_range=(1, 2), test_size=0.2):
    """
    Vectorize text data using TF-IDF with specified n-gram range.

    Parameters:
    - text_data: List of text data.
    - labels: List of corresponding labels.
    - ngram_range: Tuple specifying the range of n-grams (default is (1, 2)).
    - test_size: Fraction of the dataset to be used as the test set (default is 0.2).
    - random_state: Seed for random number generation (default is None).

    Returns:
    - Tuple (train_tfidf, test_tfidf, y_train, y_test)
      - train_tfidf: TF-IDF transformed training data.
      - test_tfidf: TF-IDF transformed test data.
      - y_train: Training labels.
      - y_test: Test labels.
    """
    # Split the data
    x_train, x_test, y_train, y_test = train_test_split(text_data, labels, test_size=test_size, random_state= 2)

    # Vectorize using TF-IDF
    vectorizer = TfidfVectorizer(ngram_range=ngram_range)
    x_train_tfidf = vectorizer.fit_transform(x_train)
    x_test_tfidf = vectorizer.transform(x_test)

    return x_train_tfidf, x_test_tfidf, y_train, y_test



In [4]:
def average_tfidf_vectors(tfidf_matrix, indices):
    """
    Computes the average TF-IDF vector for the given indices in the TF-IDF matrix.

    Parameters:
    tfidf_matrix (numpy.ndarray): The TF-IDF matrix.
    indices (list): List of indices to compute the average for.

    Returns:
    numpy.ndarray: The average TF-IDF vector.
    """
    selected_vectors = tfidf_matrix[indices]
    average_vector = np.mean(selected_vectors, axis=0)
    return average_vector